In [34]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [35]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [36]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [37]:
SYSTEM_PROMPT = """
Based on the provided use case specification, please generate a set of test scenarios to thoroughly test the functionality described in the use case.
Your task is to generate suitable test scenarios that cover all the flows and test the boundaries of conditions, if possible. Each scenario should clearly and specifically describe the situation to be tested.
Example for condition: "a valid username must be over 8 characters and below 30 characters." When you meets something like this, make sure to treat each condition as a negative scenario seperately. That means you will have two negative scenario: invalid username with lower than 8 characters and invalid username with higher than 30 characters. 
Please ensure that your test scenarios cover both the main flow and any alternate flows mentioned in the use case specification. Also, consider potential exception flows to address errors, failures, or unexpected events that could occur during the execution of the use case and encompass negative test scenarios (including edge cases and invalid inputs), and unexpected user behaviors.
Keep in mind the agile nature of the application and prioritize scenarios that align closely with the given use case specification and the application's requirements.
Return the list of test scenarios name to test the use case
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

In [38]:
usecase_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Shopping System"
save_path = r"D:\Dissertation-GPT\evaluate\prove\Shopping System\ts"

In [39]:
usecase_name_list = []
project_name = os.path.basename(usecase_path)

In [40]:
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [41]:
for usecase_name in usecase_name_list:
    print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    prompt_all = read_file_content(usecase_directlink)
    promptSYSTEM_PROMPT = [
        { "role": "system", "content": SYSTEM_PROMPT},
        { "role": "user", "content": prompt_all}
    ]
    gpt_response = ask(promptSYSTEM_PROMPT, client, model)    
    print("gpt_response \n" +gpt_response)
    write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), gpt_response)

add to cart
gpt_response 
1. Add Single Option Product to Cart Successfully
2. Add Multi-Option Product to Cart Successfully
3. Adjust Quantity Before Adding Product to Cart
4. Add Product Without Selecting Option
5. Add Product With Quantity Exceeding Stock
6. Decrease Quantity Below One
7. Add Out of Stock Product to Cart
8. Add Product With Out of Stock Option to Cart
9. View Cart After Adding Product
10. Add Product to Cart Without Being Logged In
11. Add Product to Cart With Maximum Stock Quantity
12. Add Product to Cart With Minimum Stock Quantity (1)
13. Add Multiple Different Products to Cart
14. Add Multiple Quantities of the Same Product to Cart
15. Update Cart Icon After Adding Product
16. Attempt to Add Discontinued Product to Cart
17. Add Product to Cart and Then Log Out
18. Add Product to Cart, Then Continue Shopping
19. Add Product to Cart from Wishlist
20. Add Product to Cart from Product Recommendation
String has been written to D:\Dissertation-GPT\evaluate\prove\Shopp